In [1]:
import os
try:
    os.chdir('src')
except:
    pass
print(os.getcwd())

/home/shamvinc/ssl_time_series/mvts_transformer/src


In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload


<center><img src="../img/img_0.PNG"  width="1000" height="240"/></center>


# SimMTM is a simple self-supervised learning framework for Masked Time-Series Modeling.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from tqdm import tqdm
import copy



from datasets.datasplit import split_dataset
from datasets.data import data_factory, Normalizer, TSRegressionArchive, CSVRegressionArchive
from datasets.datasplit import split_dataset
from datasets.dataset import collate_superv
from models.ts_transformer import model_factory
from models.loss import get_loss_module, contrastive_loss
from optimizers import get_optimizer

from options import Options
from running import setup


# Masked Modeling

<img src="../img/img_1.PNG"  width="900" height="240"/>

<img src="../img/img_2.PNG"  width="900" height="240"/>

### Random Masking is not a good choice to learn a good representation. 

<img src="../img/img_3.PNG" width="600"/>

### Therefore, we randomly mask a sequence of points in a serie.

In [4]:
def geom_noise_mask_single(L, lm, masking_ratio):
    """
    Randomly create a boolean mask of length `L`, consisting of subsequences of average length lm, masking with 0s a `masking_ratio`
    proportion of the sequence L. The length of masking subsequences and intervals follow a geometric distribution.
    Args:
        L: length of mask and sequence to be masked
        lm: average length of masking subsequences (streaks of 0s)
        masking_ratio: proportion of L to be masked

    Returns:
        (L,) boolean numpy array intended to mask ('drop') with 0s a sequence of length L
    """
    keep_mask = np.ones(L, dtype=bool)
    p_m = 1 / lm  # probability of each masking sequence stopping. parameter of geometric distribution.
    p_u = p_m * masking_ratio / (1 - masking_ratio)  # probability of each unmasked sequence stopping. parameter of geometric distribution.
    p = [p_m, p_u]

    # Start in state 0 with masking_ratio probability
    state = int(np.random.rand() > masking_ratio)  # state 0 means masking, 1 means not masking
    for i in range(L):
        keep_mask[i] = state  # here it happens that state and masking value corresponding to state are identical
        if np.random.rand() < p[state]:
            state = 1 - state

    return keep_mask

# SimMTM ultilizes both contrastive learning and mask modeling to learn the data representation.


# 1 - Contrastive learning

<img src="../img/img_5.png"/>

### The contrastive loss is the following: (Eq. 8 in the paper)

<center><img src="../img/img_6.PNG"/><center/>

In [5]:
def demo_contrastive_loss(z, batch_size):
    s = s.squeeze(-1) 

    B = s.shape[0]
    v = s.reshape(B, -1)

    norm_v = torch.norm(v, p=2, dim=-1).unsqueeze(-1)
    v = v/norm_v
    u = torch.transpose(v, 0, 1)

    R = torch.matmul(v,u)

 
    R = torch.exp(R/tau) # (batch + mask size) x (batch + mask size)
    
    # number of masks
    M = B//batch_size
    mask = torch.eye(batch_size, device=R.device).repeat_interleave(M,dim=0).repeat_interleave(M,dim=1)

    denom = R * (torch.ones_like(R) - torch.eye(R.shape[0], device=R.device))

    denom = R.sum(-1).unsqueeze(-1)

    loss = torch.log(R/denom)
    

    loss = (loss * (mask - torch.eye(R.shape[0], device=R.device))).sum(1)/(M-1) # except no masked unit
    loss = loss.mean(0)
    
    return -loss


# 2 - Masked Modeling

### SimMTM proposes to recover a time serie by the weighted sum of multiple masked points, which eases the reconstruction task by assembling ruined but complementary temporal variations.

<img src="../img/img_4.png"/>

In [6]:


class DemoSimMTMTransformerEncoder(nn.Module):

    def forward(self, X):
        """
        Args:
            X: (batch_size, seq_length, feat_dim) torch tensor of masked features (input)
            padding_masks: (batch_size, seq_length) boolean tensor, 1 means keep vector at this position, 0 means padding
        Returns:
            output: (batch_size, seq_length, feat_dim)
        """

        # permute because pytorch convention for transformers is [seq_length, batch_size, feat_dim]. padding_masks [batch_size, feat_dim]
        _x = X
     
        for i in range(self.temporal_unit):
            mask = geom_noise_mask_single(X.shape[0] * X.shape[1] * X.shape[2], 5, 0.3)
            mask = mask.reshape(X.shape[0], X.shape[1], X.shape[2])
            mask = torch.from_numpy(mask).to(X.device)
            x_masked = mask * X
            _x = torch.cat([_x, x_masked], axis=-1) # [batch_size, seq_length, feat_dim * temporal_unit]
    
        
        _x = _x.reshape(X.shape[0] * (self.temporal_unit + 1), X.shape[1], X.shape[2])
  

        inp = _x.permute(1, 0, 2)
        inp = self.project_inp(inp) * math.sqrt(self.d_model)  # [seq_length, batch_size, d_model] project input vectors to d_model dimensional space
        inp = self.pos_enc(inp)  # add positional encoding

        
        output = self.transformer_encoder(inp)  # (seq_length, batch_size, d_model)
        output = self.act(output)  # the output transformer encoder/decoder embeddings don't include non-linearity
        output = output.permute(1, 0, 2)  # (batch_size, seq_length, d_model)
        output = self.dropout1(output)

        z_hat = self.project(output)
        # Most probably defining a Linear(d_model,feat_dim) vectorizes the operation over (seq_length, batch_size).
        output = self.output_layer(z_hat)  # (batch_size, seq_length, feat_dim)

        return output

    
    def project(self, z, tau=0.02):
        _z = z.transpose(1, 2) # [batch_size, d_model, seq_length]
        s = self.projector_layer(_z) # [batch_size, d_model, 1]
        s = s.squeeze(-1) 

        B = s.shape[0]
        v = s.reshape(B, -1)

        norm_v = torch.norm(v, p=2, dim=-1).unsqueeze(-1)
        v = v/norm_v
        u = torch.transpose(v, 0, 1)
        
        R = torch.matmul(v,u)
     
  
        R = torch.exp(R/tau) # (batch + mask size) x (batch + mask size)
        R = R * (torch.ones_like(R) - torch.eye(R.shape[0], device=R.device)) # zero out the weight of no masked component
        R = R/R.sum(-1).unsqueeze(-1)
        M = self.temporal_unit + 1
        R = R[::M] # extract every no mask unit # (batch size) x (batch + mask size)

        z_hat = (R.unsqueeze(-1).unsqueeze(-1) * z.unsqueeze(0)).sum(1) 
        return z_hat




# Data Loading and Preparation

In [7]:
args = Options().parse()  # `argsparse` object
# args.data_dir = '../datasets/BeijingPM25Quality'
args.data_dir = '../datasets/AppliancesEnergy'
 
args.task = 'regression'
args.output_dir = '../experiments'
config = setup(args)
# config = setup(args)  # configuration dictionary

2023-08-21 00:55:10,648 | INFO : Stored configuration file in '../experiments/_2023-08-21_00-55-10_Btc'


In [8]:
data = TSRegressionArchive(config['data_dir'], pattern='TRAIN', config=config)
test_data = TSRegressionArchive(config['data_dir'], pattern='TEST', config=config)

119it [00:03, 29.85it/s]
66it [00:01, 37.69it/s] 


In [20]:
data.feature_df

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,...,dim_14,dim_15,dim_16,dim_17,dim_18,dim_19,dim_20,dim_21,dim_22,dim_23
0,-0.269570,-2.153832,-0.259319,-2.121628,-0.999057,-1.514857,-0.514935,-1.837894,-1.126190,-0.656912,...,0.267595,-1.654712,-0.454857,-0.464965,-0.418045,0.330620,-1.948513,1.202557,0.106478,-1.981361
0,-0.269570,-2.089203,-0.288860,-2.089363,-0.999057,-1.497081,-0.514935,-1.823090,-1.126190,-0.672042,...,0.234318,-1.654712,-0.454857,-0.456310,-0.436693,0.342184,-1.915849,1.134967,0.106478,-1.973379
0,-0.269570,-2.056059,-0.303630,-2.027255,-0.999057,-1.464492,-0.528758,-1.786821,-1.088955,-0.668638,...,0.199456,-1.640794,-0.454857,-0.456310,-0.455342,0.353748,-1.883185,1.067377,0.106478,-1.965397
0,-0.287474,-2.006345,-0.330216,-1.962726,-0.999057,-1.464492,-0.528758,-1.758695,-1.126190,-0.675447,...,0.151917,-1.616753,-0.472082,-0.464965,-0.473990,0.365312,-1.850521,0.999787,0.106478,-1.957415
0,-0.323280,-1.988116,-0.343509,-1.907877,-0.999057,-1.464492,-0.544116,-1.726867,-1.126190,-0.686795,...,0.123394,-1.628141,-0.472082,-0.464965,-0.492638,0.376875,-1.817857,0.932197,0.106478,-1.949433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,-0.104462,-1.923486,-0.166268,-1.946594,-0.907768,-1.286734,-0.408963,-1.883045,-1.126190,-0.545512,...,0.437149,-1.565509,-0.454857,-0.503523,-0.371424,0.307492,-2.111834,1.540506,0.106478,-2.101089
94,-0.144247,-2.054402,-0.181038,-2.034514,-0.872534,-1.464492,-0.456574,-1.853438,-1.126190,-0.577475,...,0.405457,-1.589549,-0.454857,-0.489359,-0.380749,0.312118,-2.079170,1.472917,0.106478,-2.077144
94,-0.166129,-2.130632,-0.210578,-2.113562,-0.939800,-1.545471,-0.487290,-1.899329,-1.088955,-0.592606,...,0.375349,-1.602835,-0.454857,-0.482277,-0.390073,0.316744,-2.046506,1.405327,0.106478,-2.053198
94,-0.209893,-2.213490,-0.259319,-2.146632,-0.970229,-1.556334,-0.487290,-1.862320,-1.107573,-0.618706,...,0.346826,-1.616753,-0.454857,-0.482277,-0.399397,0.321369,-2.013842,1.337737,0.106478,-2.029252


In [24]:
train_indices, val_indices, _ = split_dataset(data_indices=data.all_IDs,
                                                         validation_method='ShuffleSplit',
                                                         n_splits=1,
                                                         validation_ratio=0.2,
                                                         test_set_ratio=0,  # used only if test_indices not explicitly specified
                                                         test_indices=None,
                                                         random_seed=1337,
                                                         labels=None)
train_indices = train_indices[0]
val_indices = val_indices[0]
test_indices = np.array(test_data.all_IDs)

In [25]:
normalizer = Normalizer(config['normalization'])
data.feature_df = normalizer.normalize(data.feature_df)
test_data.feature_df = normalizer.normalize(test_data.feature_df)

In [1135]:
config['task'] = 'simmtm'
config['normalization_layer'] = 'BatchNorm'
config['out_len'] = 1
config['out_dim'] = 1
config['d_model'] = 8
config['dim_feedforward'] = 16
config['num_heads'] = 4
config['num_layers'] = 2

# SimMTMTransformerEncoder
from models.ts_transformer import model_factory
model = model_factory(config, data)
device = "cuda"
model.to(device)
model.tau = 0.05
min_tau = 0.05
model.mask_length = data.feature_df.loc[0].shape[0]//4
print(model.mask_length)
model.mask_ratio = .5

36


In [1136]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [1137]:
from torch.utils.data import DataLoader
batch_size = 64
max_len = data.feature_df.loc[0].shape[0]
train_dataloader = DataLoader(train_indices, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_indices, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_indices, batch_size=batch_size, shuffle=True)



# Pre-Training Loop

In [1138]:

i = 0
max_epoch = 0
best_loss = 1e10
best_model = copy.deepcopy(model)
best_epoch = 0
loss_fn = nn.MSELoss()


while i < max_epoch:
    train_loss = { "loss": [], "loss_mse": [], "loss_con": []}
    progress_bar = tqdm(train_dataloader)
    
    for IDs in progress_bar:
        model.train()
        X = torch.tensor(data.feature_df.loc[IDs].to_numpy()).to(device)
        X = X.float()
        X = X.reshape(-1, max_len, X.shape[-1])
        
        pred, R, w1, w2, norm = model(X)  # (batch_size, padded_length, feat_dim)
        
        loss_mse = loss_fn(pred, X) 

        loss_con = contrastive_loss(R, X.shape[0])
        
        loss_norm =  norm.mean()

        loss = 1/(w1.pow(2)) * loss_mse + 1/(w2.pow(2)) * loss_con + torch.log(w1) + torch.log(w2) 
  


        optimizer.zero_grad()
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=4.0)
        optimizer.step()

        progress_bar.set_description("Epoch {0} - Training loss: {1:.2f} - MSE loss: {2:.2f} - Contrastive loss: {3:.2f} - norm loss: {3:.2f}".format(i, 
                loss.cpu().detach().numpy().item(), loss_mse.cpu().detach().numpy().item(), 
                loss_con.cpu().detach().numpy().item(), loss_norm.cpu().detach().numpy().item())) 
        train_loss["loss"].append(loss)
        train_loss["loss_mse"].append(loss_mse)
        train_loss["loss_con"].append(loss_con)
    model.tau = max(model.tau * 0.98, min_tau)
        
    
    val_loss = { "loss": [], "loss_mse": [], "loss_con": []}
    for IDs in val_dataloader:
        model.eval()
        X = torch.tensor(data.feature_df.loc[IDs].to_numpy()).to(device)
        X = X.float()
        X = X.reshape(-1, max_len, X.shape[-1])
        
                
        pred, R, w1, w2, norm = model(X.float())  # (batch_size, padded_length, feat_dim)
        
        loss_mse = loss_fn(pred, X) 

        loss_con = contrastive_loss(R, X.shape[0])

        loss = 1/(w1.pow(2)) * loss_mse + 1/(w2.pow(2)) * loss_con + torch.log(w1) + torch.log(w2)
        
        val_loss["loss"].append(loss)
        val_loss["loss_mse"].append(loss_mse)
        val_loss["loss_con"].append(loss_con)
    
    train_loss["loss"] = torch.tensor(train_loss["loss"]).mean()
    train_loss["loss_mse"] = torch.tensor(train_loss["loss_mse"]).mean()
    train_loss["loss_con"] = torch.tensor(train_loss["loss_con"]).mean()
    val_loss["loss"] = torch.tensor(val_loss["loss"]).mean()
    val_loss["loss_mse"] = torch.tensor(val_loss["loss_mse"]).mean()
    val_loss["loss_con"] = torch.tensor(val_loss["loss_con"]).mean()
    
    if val_loss["loss"] < best_loss and model.tau == min_tau:
        best_loss = val_loss["loss"]
        best_model = copy.deepcopy(model)
        best_epoch = i
    
    progress_bar.write("Epoch {0} - Training loss: {1:.2f} {2:.2f} {3:.2f} - Validation loss: {4:.2f} {5:.2f} {6:.2f}".format(i, 
            train_loss["loss"].cpu().detach().numpy().item(), train_loss["loss_mse"].cpu().detach().numpy().item(), train_loss["loss_con"].cpu().detach().numpy().item(),
            val_loss["loss"].cpu().detach().numpy().item(), val_loss["loss_mse"].cpu().detach().numpy().item(), val_loss["loss_con"].cpu().detach().numpy().item()))
    i += 1
    
    
tqdm.write("Best Epoch {} - Best Validation loss: {}".format(best_epoch, best_loss))

Best Epoch 0 - Best Validation loss: 10000000000.0


# Replace the output layer for downstream task

In [1139]:
finetune_model = copy.deepcopy(best_model)
# finetune_model.predict_layer1 = nn.Linear(finetune_model.max_len , 1)
# out_size = data.labels_df.shape[1]
# hid_size = finetune_model.d_model 
# finetune_model.predict_layer2 = nn.Linear(hid_size , out_size)

In [1140]:


optimizer = torch.optim.AdamW(finetune_model.parameters(), lr=1e-3)

# Finetune Training Loop

In [1141]:
i = 0
max_epoch = 400
best_loss = 1e10
best_finetune_model = None
best_epoch = 0
device = "cuda"
finetune_model.to(device)
while i < max_epoch:
    train_loss = []
    progress_bar = tqdm(train_dataloader)
    
    for IDs in progress_bar:
        finetune_model.train()
        X = torch.tensor(data.feature_df.loc[IDs].to_numpy()).to(device)
        X = X.reshape(-1, max_len, X.shape[-1])
        targets = torch.tensor(data.labels_df.loc[IDs].to_numpy()).to(device)
        pred = finetune_model.predict(X.float()).squeeze(-1)
        loss = loss_fn(pred, targets)


        optimizer.zero_grad()
        loss.backward()

        nn.utils.clip_grad_norm_(finetune_model.parameters(), max_norm=4.0)
        optimizer.step()

        progress_bar.set_description("Epoch {} - Training loss: {}".format(i, loss)) 
        train_loss.append(loss)
    
    val_loss = []
    for IDs in val_dataloader:
        finetune_model.eval()
        X = torch.tensor(data.feature_df.loc[IDs].to_numpy()).to(device)
        X = X.reshape(-1, max_len, X.shape[-1])
        targets = torch.tensor(data.labels_df.loc[IDs].to_numpy()).to(device)
        pred = finetune_model.predict(X.float()).squeeze(-1)
        loss = loss_fn(pred, targets)
        val_loss.append(loss)
    
    train_loss = torch.tensor(train_loss).mean()
    val_loss = torch.tensor(val_loss).mean()
    
    if val_loss < best_loss:
        best_loss = val_loss
        best_finetune_model = copy.deepcopy(finetune_model)
        best_epoch = i
    
    progress_bar.write("Epoch {} - Training loss: {} - Validation loss: {}".format(i, train_loss, val_loss))
    i += 1
    
    
tqdm.write("Best Epoch {} - Best Validation loss: {}".format(best_epoch, best_loss))

Epoch 4 - Training loss: 183.98291015625: 100%|███████████████████████████████████████| 2/2 [00:00<00:00, 51.66it/s]

Epoch 0 - Training loss: 186.9484100341797 - Validation loss: 269.57745361328125
Epoch 1 - Training loss: 219.54864501953125 - Validation loss: 268.97491455078125
Epoch 2 - Training loss: 207.2030792236328 - Validation loss: 268.5617370605469
Epoch 3 - Training loss: 182.6446533203125 - Validation loss: 268.2022399902344



Epoch 8 - Training loss: 184.15869140625: 100%|███████████████████████████████████████| 2/2 [00:00<00:00, 51.39it/s]

Epoch 4 - Training loss: 194.3675994873047 - Validation loss: 267.8271484375
Epoch 5 - Training loss: 188.67726135253906 - Validation loss: 267.3499450683594
Epoch 6 - Training loss: 212.71441650390625 - Validation loss: 266.77508544921875
Epoch 7 - Training loss: 199.86184692382812 - Validation loss: 266.00897216796875



Epoch 12 - Training loss: 167.44949340820312: 100%|███████████████████████████████████| 2/2 [00:00<00:00, 52.87it/s]

...

Epoch 388 - Training loss: 12.12100601196289 - Validation loss: 16.34530258178711
Epoch 389 - Training loss: 13.381879806518555 - Validation loss: 16.978267669677734
Epoch 390 - Training loss: 8.723334312438965 - Validation loss: 18.012990951538086
Epoch 391 - Training loss: 10.646793365478516 - Validation loss: 18.47659683227539
Epoch 392 - Training loss: 8.700498580932617 - Validation loss: 18.167831420898438


  0%|                                                                                         | 0/2 [00:00<?, ?it/s]

Epoch 393 - Training loss: 8.662198066711426 - Validation loss: 18.268199920654297
Epoch 394 - Training loss: 12.334908485412598 - Validation loss: 18.231468200683594
Epoch 395 - Training loss: 12.585148811340332 - Validation loss: 17.7148494720459
Epoch 396 - Training loss: 8.049286842346191 - Validation loss: 17.236398696899414
Epoch 397 - Training loss: 15.363054275512695 - Validation loss: 16.445581436157227


Epoch 399 - Training loss: 11.325026512145996: 100%|██████████████████████████████████| 2/2 [00:00<00:00, 47.94it/s]

Epoch 398 - Training loss: 10.398223876953125 - Validation loss: 16.20722198486328
Epoch 399 - Training loss: 12.503231048583984 - Validation loss: 15.918633460998535
Best Epoch 177 - Best Validation loss: 14.74407958984375


In [1142]:
test_loss = []
for IDs in test_dataloader:
    best_finetune_model.eval()
    X = torch.tensor(test_data.feature_df.loc[IDs].to_numpy()).to(device)
    X = X.reshape(-1, max_len, X.shape[-1])
    targets = torch.tensor(test_data.labels_df.loc[IDs].to_numpy()).to(device)
    pred = best_finetune_model.predict(X.float()).squeeze(-1)
    loss = loss_fn(pred, targets)
    test_loss.append(loss)


test_loss = torch.tensor(test_loss).mean()
print("Test loss: {}".format(test_loss))
print("Test loss: {}".format(np.sqrt(test_loss)))

Test loss: 20.3616943359375
Test loss: 4.512393474578857


# Forecasting Results from the paper

<center><img src="../img/img_7.PNG"/><center/>

Reference:
1. https://arxiv.org/abs/2302.00861
2. https://github.com/gzerveas/mvts_transformer

No Pretrain

Test loss: 17.563016891479492

Test loss: 24.951457977294922

Test loss: 13.356301307678223

Test loss: 15.543647766113281

Test loss: 10.70577621459961

Test loss: 11.653989791870117

Test loss: 28.348876953125

Test loss: 9.66756534576416

Test loss: 22.540130615234375

Test loss: 20.3616943359375

Pretrain L//2 pretrain epoch 50

Test loss: 11.152569770812988

Test loss: 8.513360023498535

Test loss: 20.0480899810791

Test loss: 12.694931983947754

Test loss: 15.458633422851562

Test loss: 16.706832885742188

Test loss: 11.432455062866211

Test loss: 7.263433456420898

Test loss: 15.908415794372559

Test loss: 17.736656188964844

Pretrain L//2 pretrain epoch 20

Test loss: 13.188956260681152

Test loss: 12.610322952270508

Test loss: 22.47460174560547

Test loss: 17.15345001220703

Test loss: 20.823780059814453

Pretrain L//2 pretrain epoch 100

Test loss: 20.534494400024414

Test loss: 25.885923385620117

Test loss: 21.357059478759766

Test loss: 20.61741828918457

Test loss: 24.453622817993164

Pretrain L//4 pretrain epoch 50

Test loss: 8.65383243560791

Test loss: 7.952672481536865

Test loss: 13.125472068786621

Test loss: 17.975181579589844

Test loss: 21.050844192504883

Test loss: 9.025917053222656

Test loss: 13.526683807373047

Test loss: 16.005821228027344

Test loss: 23.140092849731445

Test loss: 11.915889739990234

Pretrain L//10 pretrain epoch 50

Test loss: 18.413068771362305

Test loss: 17.07589340209961

Test loss: 14.856538772583008

Test loss: 9.033221244812012

Test loss: 16.976844787597656